In [2]:
using DelimitedFiles
using CSV
using DataFrames
data = CSV.read("C:\\Users\\nickh\\vectorized_xml_vectors_50_sorted", DataFrame, header=0)
#readdlm("C:\\Users\\nickh\\vectorized_xml_vectors_50", " ", Float64)
#[Mu,Cov,P,Pi,LL,TL,gamma]=pmhmm(reshape(reshape(a', [], size(a)(2)), 50,[])', size(a)(2) / 50, 3,4)

N = size(data)[1]
D = 50
#T = Int((size(data)[2] - 1) / D)
T = 10
dfa = Array(coalesce.(DataFrame(data)[:, 2:T*D+1], 0)) #, N*T

raw = zeros(N*T, D)

for n=1:N
    for t=1:T
        raw[((n-1)*T)+t,:] .= dfa[n, (t-1)*D+1:t*D]
    end
end
size(dfa)

┌ Warning: thread = 1 warning: only found 1351 / 1451 columns around data row: 2. Filling remaining columns with `missing`
└ @ CSV C:\Users\nickh\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1351 / 1451 columns around data row: 3. Filling remaining columns with `missing`
└ @ CSV C:\Users\nickh\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1351 / 1451 columns around data row: 4. Filling remaining columns with `missing`
└ @ CSV C:\Users\nickh\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1351 / 1451 columns around data row: 5. Filling remaining columns with `missing`
└ @ CSV C:\Users\nickh\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1301 / 1451 columns around data row: 6. Filling remaining columns with `missing`
└ @ CSV C:\Users\nickh\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1301 / 1451 column

┌ Warning: thread = 1 warning: only found 1051 / 1451 columns around data row: 56. Filling remaining columns with `missing`
└ @ CSV C:\Users\nickh\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1051 / 1451 columns around data row: 57. Filling remaining columns with `missing`
└ @ CSV C:\Users\nickh\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1051 / 1451 columns around data row: 58. Filling remaining columns with `missing`
└ @ CSV C:\Users\nickh\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1051 / 1451 columns around data row: 59. Filling remaining columns with `missing`
└ @ CSV C:\Users\nickh\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1051 / 1451 columns around data row: 60. Filling remaining columns with `missing`
└ @ CSV C:\Users\nickh\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1051 / 1451 c

(6223, 500)

In [3]:
include("FHMMs.jl")

K=3
M=3
#Y = rand(N*T, D) # NT*D

model = Main.FHMMs.StructuredFHMM(raw, M,K,D,T,N)
Main.FHMMs.fit_structured!(model, raw, 20,100,1.0e-8)

"Variational converged in 17 iterations"

"Step 1: Likelihood 0"

"Step 2: Likelihood -6.030002213428542e6"

"Variational converged in 5 iterations"

"Step 3: Likelihood -2.281061499725315e6"

"Variational converged in 4 iterations"

"Step 4: Likelihood -2.2187301340685943e6"

"Variational converged in 4 iterations"

"Step 5: Likelihood -2.157758963550973e6"

"Converged after 5 iterations"

In [5]:
using Distances
using Distributions
using Statistics
# T must divide nwidth 
nwidth = 1
numrows=Int(size(model.γ)[1] / nwidth)
γ_avg = zeros(numrows, size(model.γ)[2])
display("numrows $numrows")
for i=1:numrows
    γ_avg[i,:] = mean(model.γ[(i-1)*nwidth+1:(i*nwidth),:],dims=1)
end

γ_avg = reshape(γ_avg, size(dfa)[1], :)
size(γ_avg)

"numrows 62230"

(6223, 90)

In [9]:


idx=rand(1:size(γ_avg)[1])
display(idx)
display(data[idx,1])
#[vals, indices]=sort(pdist2(gamma_r(idx,:), gamma_r))
display(size(pairwise(Euclidean(), reshape(γ_avg[idx,:], :, 1), γ_avg', dims=2)))
indices = sortperm(vec(pairwise(Euclidean(), reshape(γ_avg[idx,:], :, 1), γ_avg', dims=2)))
display(data[indices[1],1])
display(data[indices[2],1])

indices = sortperm(vec(pairwise(Euclidean(), reshape(dfa[idx,:], :, 1), dfa', dims=2)))
display(data[indices[1],1])
display(data[indices[2],1])

73

"检查好，你什么时候看到稻草人，想清楚了吗？"

(1, 6223)

"检查好，你什么时候看到稻草人，想清楚了吗？"

"你必须把你的泳衣并与我游泳。"

"检查好，你什么时候看到稻草人，想清楚了吗？"

"没什么，没什么就是没什么"

In [ ]:
1

In [ ]:
using StatsBase
using LinearAlgebra
using Distributions
include("FHMMs.jl")

K=3
M=2
D=2
T=2

pi = [0.1 0.3; 0.8 0.2; 0.1 0.5]

C = [ 0.2 0.1; 0.1 0.2 ]

P = zeros(K,K,M)
P[:,:,1] = [ 0.25 0.35 0.3; 0.15 0.2 0.65; 0.15 0.65 0.2 ]
P[:,:,2] = [ 0.99 0.005 0.005; 0.7 0.2 0.1; 0.3 0.3 0.4 ]

W = zeros(D,K,M)
W[:,:,1] = [0.5 -1 99; -0.5 -1 99]
W[:,:,2] = [50 0 1; -50 0 1]

model = Main.FHMMs.FHMM(D,K,M,    W, pi, P, C)

#Y_n, states_n = generate(model, T)
#display(Y)
Y = [ 0.210517 48.5192; -0.219267 -51.817]

h, γ, ξ = Main.FHMMs.fit_sv2_start!(model, Y)

In [ ]:
display(model.P)

In [ ]:
for i=1:1000
    Y_n, states_n = generate(model, T)
    h, γ, ξ = Main.FHMMs.fit_sv2!(model, Y_n, h, γ, ξ)
end

In [ ]:


function generate(model::Main.FHMMs.FHMM, timesteps::Int)
    X = zeros(size(model.W)[1], timesteps)
    # choose initial states based on π
    states = zeros(Int64, model.M, timesteps)
    for m=1:model.M
        idx = sample(weights(model.π[:,m]))
        states[m,1] = Int64(idx)
    end
    for t=1:timesteps    
        mu = zeros(model.D)
        for m=1:model.M
            mu += model.W[:,states[m,t],m]
        end
        n = MvNormal(mu, Array{Float64,2}(Hermitian(Symmetric(model.C))))
        X[:,t] = rand(n)
        if t < timesteps
            for m=1:model.M
                nsp = model.P[states[m,t],:,m]
                smpl = sample(weights(nsp))
                states[m,t+1] = smpl
            end
        end
    end
    return X,states
end

In [ ]:
function sample_states(model::Main.FHMMs.FHMM, timesteps::Int, iter=1000)
    # we need to sample theta, KxMxT
    # θ[1,1,2] represents the probabilities of chain (1) being in state (1) at time 2, 
    # θ[3,1,2] represents the probabilities of chain (1) being in state (3) at time 2, etc
    # this is unconditional wrt the observation
    theta_sampled = zeros(model.K,model.M, timesteps)
    for i=1:iter
        theta_iter = zeros(model.K,model.M,timesteps)
        for m=1:model.M
            s = sample(weights(model.π[:,m]))
            theta_iter[s, m,1] += 1
        end
        for t=2:timesteps
            for m=1:model_actual.M
                lastState = findfirst((t) -> t == 1, theta_iter[:,m,t-1])
                s = sample(weights(model.P[lastState,:,m]))
                theta_iter[s,m,t] += 1
            end
        end
        theta_sampled .+= theta_iter
    end
    theta_sampled ./= iter
    return theta_sampled
end


In [ ]:
function get_state_combinations(M::Int, K::Int)
    state_combinations = []
    m = 1
    stack = []
    while(true)
        if(m == M)
            if(size(stack)[1] != M - 1)
                break
            end
            for s=1:K
                push!(stack, s)
                push!(state_combinations, copy(stack))
                pop!(stack)
            end
            stack[end] +=1
            while(stack[end] > K)
                pop!(stack)
                if(size(stack)[1] == 0)
                    break
                end
                stack[end] +=1
                m -= 1
            end
        else
            push!(stack, 1)
            m += 1
        end
    end
    return state_combinations
end

In [ ]:
# we want to calculate 
# P(Mt=St|Yt)
# = P(Yt|Mt=St).P(Mt=St|Mt-1=St-1) / P(Yt)
# This decoding is specific to the completely factorized VI model which uses a simplified P(Mt|Yt) that assumes 
# no dependence between timesteps. Viterbi decoding is therefore not required, since we don't need to find the max sequence,
# we just take the max P at each timestep.
# even though each chain is independent but the observation is conditional on *all* chains 
# this is because Yt is a sum of M1,M2,M3...
# i.e. P(Yt|M1t=S1t,M2t=.., M3t=..) != P(Yt|M1t=S1t)
# but we have θ, which is an estimation of P(M1t=S1t), P(M1t=S2t)...P(Mmt=Sst)
# so we need to find all configurations of M/S at this t
# cross product of (M1xS)x(M2xS)x..x(MkxS)
# if we are using the variational inference model, θ is KxMxT
# θ[1,1,2] represents p(chain (1) in state (1) at time 2) 
# θ[3,1,2] represents p(chain (1) in state (3) at time 2), etc
# this is unconditional wrt the observation
# i.e. P(Mt=St|M(t-1)=S(t-1))
# therefore P(Mt=St) is the product over dim(1) at T
function decode(model::Main.FHMMs.FHMM, X::Array{Float64,2}, θ::Array{Float64,3})
    
    D=size(X)[1]
    timesteps=size(X)[2]
    T = timesteps
    
    states = zeros(model.M,timesteps)
    for t=1:timesteps
        pmax = -1
        mmax = []
        stack = []
        m = 1
        M = model.M
        K = model.K
        while(true)
            if(m == M)
                if(size(stack)[1] != M - 1)
                    break
                end
                for s=1:K
                    push!(stack, s)
                    mu = zeros(D)
                    for i=1:M
                        mu += model.W[:, stack[i], i]
                    end
                    
                    p_yt_given_mt_in_s = pdf(MvNormal(mu, Array{Float64,2}(Hermitian(Symmetric(model.C)))), X[:,t])
                    p_mt_in_s = reduce(*, θ[stack,:,t])
                    unnormalized = p_yt_given_mt_in_s * p_mt_in_s
                    if unnormalized > pmax
                        pmax = unnormalized
                        mmax = copy(stack)
                    end
                    pop!(stack)
                end
                stack[end] +=1
                while(stack[end] > K)
                    pop!(stack)
                    if(size(stack)[1] == 0)
                        break
                    end
                    stack[end] +=1
                    m -= 1
                end
            else
                push!(stack, 1)
                m += 1
            end
        end
        states[:,t] = mmax
    end
    return states
end

In [ ]:
function decode_full(model::Main.FHMMs.FHMM, X::Array{Float64,2})
    D=size(X)[1]
    timesteps=size(X)[2]
    state_combinations = get_state_combinations(model.M, model.K)
    state_probabilities = zeros(size(state_combinations)[1],timesteps)
    
    M = model.M
    K = model.K
    logP = log.(model.P)
    for t=1:timesteps
        for s_index=1:size(state_combinations)[1]
            state = state_combinations[s_index]
            mu = zeros(D)
            for i=1:model.M
                mu += model.W[:, state[i], i]
            end
            p_yt_given_mt_in_s = pdf(MvNormal(mu, Array{Float64,2}(Hermitian(Symmetric(model.C)))), X[:,t])
                   
            if(t == 1)
               p_mt_in_s = reduce(*, model.π[[CartesianIndex(state[i], i)  for i=1:model.M]])
            else
                # calculate the probability that chain M is in state K, P(Mt)
                # this is P(Mt|Mt-1, Yt), so P(Mt-1|Yt) * P(Mt|Mt-1)
                # P(Mt-1|Yt) is sum over K
                p_mt_in_s = 0
                for sc_last_index=1:size(state_combinations)[1]
                    sc_last = state_combinations[sc_last_index]
                    for m_last=1:M
                        for m_this=1:M
                            p_mt_in_s += exp(
                                state_probabilities[sc_last_index,t-1] + 
                                logP[sc_last[m_this], state[m_this], m_this])
                        end
                    end
                end
            end
            state_probabilities[s_index,t] = log(p_yt_given_mt_in_s) +  log(p_mt_in_s)
         end
    end
    max_probs = zeros(M, timesteps)
    i = 1
    for p in argmax(state_probabilities, dims=1)
        max_probs[:, p[2]] = state_combinations[p[1]]
        i += 1
    end
    return max_probs
end


In [ ]:
# setup our known model
# 
D=6
K=3
M=2
#T=5
W=cat( [ 1.0 49.0; -1.0 -49.0 ],[ 1.0 49.0; -1.0 -49.0 ], [ 1.0 49.0; -1.0 -49.0 ], dims=3)
π = [ 0.35 0.25 0.95; 0.65 0.75 0.05; ]
P = cat([ 0.3 0.7 ; 0.2 0.8 ; ],
    [ 0.5 0.5 ; 0.4 0.6 ;  ],
    [ 0.75 0.25 ; 0.89 0.11 ; ], dims=3)
W = rand(D,K,M)
π = rand(K,M)
π ./= sum(π,dims=(1))
P = rand(K, K, M)
P ./= sum(P, dims=2)
#C = rand(D,D)
C = Array{Float64}(Diagonal([0.1 for x in 1:D]))
model_actual = Main.FHMMs.FHMM(D,K,M,
    W, π, P, C)

X, actual = generate(model_actual, 4)



In [ ]:
#Y = rand(5,4)

Main.FHMMs.fit_sv2!(model_actual, X)

In [ ]:
actual

In [ ]:
X, actual = generate(model_actual, 4)
model_complete = Main.FHMMs.FHMM(D,K,M)
result = Main.FHMMs.fit!(model_complete,X,maxiter=5000,verbose=true)    
decode_full(model_complete, X)

In [ ]:
actual

In [ ]:
timesteps=10
num_observations = 100

theta_sampled = sample_states(model_actual, timesteps, 1000)
j = 0
for i in 1:num_observations
    X,actual_states = generate(model_actual, timesteps)
    estimated_states = decode(model_actual,X, theta_sampled)
    j += sum(actual_states .== estimated_states)
end
j / (num_observations * timesteps * model_actual.M)

In [ ]:
j = 0
for i in 1:num_observations
    X,actual_states = generate(model_actual, timesteps)
    estimated_states = decode_full(model_actual,X)
    j += sum(actual_states .== estimated_states)
end
j / (num_observations * timesteps * model_actual.M)

In [ ]:
model_learn = Main.FHMMs.FHMM(D,K,M)
j = 0
num_observations = 50
for i=1:num_observations
    X,actual_states = generate(model_actual, timesteps)
    
    result = Main.FHMMs.fit!(model_learn,X,maxiter=100)    
    
    estimated_states = decode_full(model_learn,X)
    j += sum(actual_states .== estimated_states)
end
j / (num_observations * timesteps * model_actual.M)

In [ ]:
timesteps=10
model_learn = Main.FHMMs.FHMM(D,K,M)
j = 0
num_observations = 100
for i=1:num_observations
    X,actual_states = generate(model_actual, timesteps)
    result = Main.FHMMs.fit!(model_learn,X,maxiter=100,fzero=true)    
    for i in 1:D
        model_learn.C[i, i] += 0.0001
    end
    #println(cholesky(Hermitian(Symmetric(model_learn.C))))
    estimated_states = decode(model_learn,X, result.θ)
    j += sum(actual_states .== estimated_states)
    println(j / (i * timesteps * model_actual.M))
end
j / (num_observations * timesteps * model_actual.M)